In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
r = requests.get("https://www.lsa.umich.edu/cg/cg_subjectlist.aspx?termArray=f_20_2310&cgtype=ug&allsections=true")

In [4]:
soup = BeautifulSoup(r.text)

In [11]:
departments = [cell.text.strip() for cell in soup.select_one(".table.table-striped.table-condensed").select("tr > td:nth-child(1)")]

In [87]:
ug_course_sections = []

In [90]:
url = "https://www.lsa.umich.edu/cg/cg_results.aspx?termArray=f_20_2310&cgtype=ug&department={}&allsections=true&show=500"
for dep in departments:
    print(dep)
    r = requests.get(url.format(dep))
    soup = BeautifulSoup(r.text)
    for result in soup.select(".row.result"):
        dept, number, _, name = [a.strip() for a in result.select_one("font").text.strip().split("\r\n")]
        section, term, credits, mode, instructor, _ = [a.text.strip() for a in result.select_one(".bottompadding_main").find_all("div", recursive=False)]
        section = " ".join(section.split())
        term = " ".join(term.split()[1:])
        credits = credits.split()[-1]
        mode = mode.split()[-1]
        instructor = " ".join(instructor.split()[1:])
        ug_course_sections.append({
            "dept": dept,
            "number": number,
            "name": name,
            "section": section,
            "term": term,
            "credits": credits,
            "mode": mode,
            "instructor": instructor,
        })

AAS
AERO
AEROSP
AES
ALA
AMCULT
ANATOMY
ANTHRARC
ANTHRBIO
ANTHRCUL
APPPHYS
ARABAM
ARABIC
ARCH
ARMENIAN
ARTDES
ARTSADMN
ASIAN
ASIANLAN
ASIANPAM
ASTRO
AUTO
BA
BCS
BIOINF
BIOLCHEM
BIOLOGY
BIOMEDE
BIOPHYS
BIOSTAT
CATALAN
CEE
CHE
CHEM
CJS
CLARCH
CLCIV
CLIMATE
CMPLXSYS
COGSCI
COMM
COMP
COMPLIT
CSP
CZECH
DANCE
DATASCI
DIGITAL
DUTCH
EARTH
EAS
ECON
EDCURINS
EDUC
EEB
EECS
ELI
ENGLISH
ENGR
ENS
ENSCEN
ENVIRON
ES
ESENG
FRENCH
FTVM
GEOG
GERMAN
GREEK
GREEKMOD
GTBOOKS
HEBREW
HISTART
HISTORY
HONORS
HS
HUMGEN
INSTHUM
INTLSTD
INTMED
IOE
ISLAM
ITALIAN
JAZZ
JUDAIC
KINESLGY
KRSTD
LACS
LATIN
LATINOAM
LING
LSWA
MACROMOL
MATH
MATSCIE
MCDB
MECHENG
MEDCHEM
MELANG
MEMS
MENAS
MFG
MICRBIOL
MIDEAST
MILSCI
MKT
MOVESCI
MUSEUMS
MUSICOL
MUSMETH
MUSTHTRE
NATIVEAM
NAVARCH
NAVSCI
NERS
NURS
ORGSTUDY
PAT
PATH
PERSIAN
PHARMACY
PHARMSCI
PHIL
PHRMACOL
PHYSICS
PHYSIOL
PIBS
POLISH
POLSCI
PORTUG
PPE
PSYCH
PUBHLTH
PUBPOL
QMSS
RCARTS
RCASL
RCCORE
RCHUMS
RCIDIV
RCLANG
RCMUSIC
RCNSCI
RCSSCI
REEES
RELIGION
ROMLANG
ROMLING
RUSSIAN
SCAND


In [91]:
len(ug_course_sections)

2454

In [92]:
import pandas as pd

In [93]:
df = pd.DataFrame(ug_course_sections)

In [98]:
df["mode"].value_counts() / len(df)

Online    0.637734
Person    0.232681
Hybrid    0.129584
Name: mode, dtype: float64

In [99]:
df.to_csv("./undergrad_sections.csv", index=False)